In [ ]:
!pip uninstall -y qiskit qiskit-terra qiskit-aer qiskit-ibm-runtime torch torchvision torchaudio

!pip install 'qiskit==2.1.1' --no-cache-dir
!pip install 'qiskit-aer==0.17.1' --no-cache-dir
!pip install 'qiskit-ibm-runtime==0.40.1' --no-cache-dir
!pip install 'qiskit-addon-utils==0.1.1' --no-cache-dir

!pip install 'torch==2.4.1' --index-url https://download.pytorch.org/whl/cpu --no-cache-dir
!pip install 'gymnasium==0.29.1' 'numpy==1.26.4' 'scipy==1.13.1' 'matplotlib==3.9.2' --no-cache-dir

!pip install mitiq==0.41.0 --no-cache-dir



print("Installation complete.")


Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 295.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 398.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 296.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 281.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 431.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 372.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 kB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 192.3 MB/s eta 0:00:00
Installation complete.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random
import time
import warnings
warnings.filterwarnings('ignore')

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter
from qiskit.transpiler import CouplingMap

from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer.primitives import EstimatorV2 as AerEstimatorV2

from qiskit.primitives import StatevectorEstimator

import mitiq
from mitiq import zne
from mitiq.zne.scaling import fold_gates_at_random
from mitiq.zne.inference import RichardsonFactory

import torch
import torch.nn as nn
import torch.nn.functional as F

import gymnasium as gym
from gymnasium import spaces

from scipy.optimize import curve_fit

print("All imports successful.")


All imports successful.


In [ ]:
class HamiltonianMirroredGenerator:
    def __init__(self, seed=42):
        self.rng = np.random.RandomState(seed)

    def generate_base_circuit(self, num_qubits=4, trotter_steps=2):
        layout = [(i, i+1) for i in range(num_qubits-1)]
        cmap = CouplingMap(layout)

        ham = generate_xyz_hamiltonian(
            cmap,
            coupling_constants=(
                self.rng.uniform(0.1, 0.5),
                self.rng.uniform(0.1, 0.5),
                self.rng.uniform(0.1, 0.5)
            ),
            ext_magnetic_field=(
                self.rng.uniform(0.05, 0.2),
                self.rng.uniform(0.05, 0.2),
                self.rng.uniform(0.05, 0.2)
            )
        )

        evo_gate = PauliEvolutionGate(
            ham,
            time=self.rng.uniform(0.2, 1.0),
            synthesis=LieTrotter(reps=trotter_steps)
        )

        circuit = QuantumCircuit(num_qubits)
        circuit.append(evo_gate, range(num_qubits))
        return circuit.decompose().decompose()

    def mirror_and_cliffordize(self, circuit):
        mirrored = circuit.copy()
        mirrored.compose(circuit.inverse(), inplace=True)

        cliff = QuantumCircuit(circuit.num_qubits)
        for inst in mirrored:
            g = inst.operation
            qb = inst.qubits
            if g.name in ['h', 'x', 'y', 'z', 's', 'sdg', 'cx', 'cz']:
                cliff.append(g, qb)
            elif g.name in ['rz', 'p', 'u1']:
                cliff.s(qb[0])
            elif g.name in ['rx', 'u2']:
                cliff.h(qb[0])
            elif g.name in ['ry', 'u3']:
                cliff.s(qb[0])
                cliff.h(qb[0])
        return cliff

gen = HamiltonianMirroredGenerator(seed=42)
base_u = gen.generate_base_circuit(4, 2)
obs = SparsePauliOp(
    ['I'*i + 'Z' + 'I'*(4-1-i) for i in range(4)],
    coeffs=[1.0/4]*4
)
print(f"Base U: {len(base_u)} gates")
print(f"Observable: average Z")


Base U: 126 gates
Observable: average Z


In [ ]:
class HamiltonianMirroredGenerator:
    def __init__(self, seed=42):
        self.rng = np.random.RandomState(seed)

    def generate_base_circuit(self, num_qubits=4, trotter_steps=2):
        layout = [(i, i+1) for i in range(num_qubits-1)]
        cmap = CouplingMap(layout)

        ham = generate_xyz_hamiltonian(
            cmap,
            coupling_constants=(
                self.rng.uniform(0.1, 0.5),
                self.rng.uniform(0.1, 0.5),
                self.rng.uniform(0.1, 0.5)
            ),
            ext_magnetic_field=(
                self.rng.uniform(0.05, 0.2),
                self.rng.uniform(0.05, 0.2),
                self.rng.uniform(0.05, 0.2)
            )
        )

        evo_gate = PauliEvolutionGate(
            ham,
            time=self.rng.uniform(0.2, 1.0),
            synthesis=LieTrotter(reps=trotter_steps)
        )

        circuit = QuantumCircuit(num_qubits)
        circuit.append(evo_gate, range(num_qubits))
        return circuit.decompose().decompose()

    def mirror_and_cliffordize(self, circuit):
        mirrored = circuit.copy()
        mirrored.compose(circuit.inverse(), inplace=True)

        cliff = QuantumCircuit(circuit.num_qubits)
        for inst in mirrored:
            g = inst.operation
            qb = inst.qubits
            if g.name in ['h', 'x', 'y', 'z', 's', 'sdg', 'cx', 'cz']:
                cliff.append(g, qb)
            elif g.name in ['rz', 'p', 'u1']:
                cliff.s(qb[0])
            elif g.name in ['rx', 'u2']:
                cliff.h(qb[0])
            elif g.name in ['ry', 'u3']:
                cliff.s(qb[0])
                cliff.h(qb[0])
        return cliff

gen = HamiltonianMirroredGenerator(seed=42)
base_u = gen.generate_base_circuit(4, 2)
obs = SparsePauliOp(
    ['I'*i + 'Z' + 'I'*(4-1-i) for i in range(4)],
    coeffs=[1.0/4]*4
)
print(f"Base U: {len(base_u)} gates")
print(f"Observable: average Z")


Base U: 126 gates
Observable: average Z


In [ ]:
NOISE_LEVEL = 0.02
SHOTS = 4096

nm = NoiseModel()
err1 = depolarizing_error(NOISE_LEVEL, 1)
err2 = depolarizing_error(NOISE_LEVEL * 2, 2)
nm.add_all_qubit_quantum_error(err1, ['h','x','y','z','s','sdg'])
nm.add_all_qubit_quantum_error(err2, ['cx','cz'])

backend_ideal = AerSimulator()
backend_noisy = AerSimulator(noise_model=nm)

def executor_ideal(circuit, observable):
    from qiskit_aer.primitives import Estimator
    est = Estimator()
    job = est.run(circuits=[circuit], observables=[observable], shots=SHOTS)
    return job.result().values[0]

def executor_noisy(circuit, observable):
    from qiskit_aer.primitives import Estimator
    est = Estimator(backend_options={"noise_model": nm})
    job = est.run(circuits=[circuit], observables=[observable], shots=SHOTS)
    return job.result().values[0]

print("Executors ready (observable parameter added)")


Executors ready (observable parameter added)


In [ ]:
class ZNEEnvironment(gym.Env):
    def __init__(self, generator):
        super().__init__()
        self.generator = generator

        self.noisefactor_options = [
            [1.0, 1.1, 1.2], [1.0, 1.2, 1.3], [1.0, 1.3, 1.5], [1.0, 1.4, 1.6],
            [1.0, 1.5, 1.8], [1.0, 1.7, 1.9], [1.0, 2.0, 2.5], [1.0, 2.2, 2.7],
            [1.0, 2.5, 2.9], [1, 2, 3], [1, 2, 4], [1, 3, 5],
            [1.0, 3.0, 3.5], [1.0, 3.2, 3.8], [1.0, 3.5, 3.9], [1.0, 4.0, 5.0],
            [1.0, 4.5, 5.0], [1, 3, 5, 7], [1, 5, 7], [1.0, 6.0, 7.0],
            [1, 1.5, 2.5, 4], [1, 2, 3, 5], [1, 3, 5, 7], [1, 1.2, 2.5, 5],
            [1, 1.5, 3, 7], [1.0, 1.8, 2.5], [1.0, 2.0, 3.5], [1.0, 3.0, 5.0],
            [1.0, 2.5, 4.0], [1.0, 3.5, 6.0]
        ]

        self.extrapolators = ['linear', 'polynomial', 'exponential', 'richardson']

        n_actions = len(self.noisefactor_options) * len(self.extrapolators)
        self.action_space = spaces.Discrete(n_actions)
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(6,), dtype=np.float32)

        self.base_u = None
        self.num_qubits = 4
        self.trotter_steps = 2

    def reset(self, seed=None, options=None):
        if seed is not None:
            self.generator.rng = np.random.RandomState(seed)

        nq = self.num_qubits
        trotter = self.trotter_steps

        self.base_u = self.generator.generate_base_circuit(nq, trotter)
        self.circuit = self.generator.mirror_and_cliffordize(self.base_u)
        self.observable = SparsePauliOp(
            ['I'*i + 'Z' + 'I'*(nq-1-i) for i in range(nq)],
            coeffs=[1.0/nq]*nq
        )

        ops = self.circuit.count_ops()
        self.features = {
            'qubits': nq,
            'cx_gates': ops.get('cx', 0) + ops.get('cz', 0),
            'single_gates': sum(ops.get(g, 0) for g in ['h','x','y','z','s','sdg']),
            'depth': self.circuit.depth(),
            'total_gates': sum(ops.values()),
            'ham_terms': 3*nq + 3*(nq-1)
        }

        self.ideal = executor_ideal(self.circuit, self.observable)

        state = np.array([
            self.features['qubits'],
            self.features['cx_gates'],
            self.features['single_gates'],
            self.features['depth'],
            self.features['total_gates'],
            self.features['ham_terms']
        ], dtype=np.float32)

        return state, {}

    def step(self, action):
        template_idx = action // len(self.extrapolators)
        extrap_idx = action % len(self.extrapolators)

        scales = self.noisefactor_options[template_idx]
        extrap_name = self.extrapolators[extrap_idx]

        folded_circs = [fold_u_then_mirror_cliff(self.base_u, s, self.generator) for s in scales]
        noisy_vals = [executor_noisy(fc, self.observable) for fc in folded_circs]

        if extrap_name == 'linear':
            mitigated = linear_extrapolate(scales, noisy_vals)
        elif extrap_name == 'polynomial':
            mitigated = polynomial_extrapolate(scales, noisy_vals, 2)
        elif extrap_name == 'exponential':
            mitigated = exponential_extrapolate(scales, noisy_vals)
        else:
            mitigated = richardson_extrapolate(scales, noisy_vals)

        error = abs(mitigated - self.ideal)
        reward = -error

        state = np.array([
            self.features['qubits'],
            self.features['cx_gates'],
            self.features['single_gates'],
            self.features['depth'],
            self.features['total_gates'],
            self.features['ham_terms']
        ], dtype=np.float32)

        info = {'factors': scales, 'extrapolator': extrap_name, 'error': error}
        return state, reward, True, False, info

env = ZNEEnvironment(gen)
print(f"Environment ready: {env.action_space.n} actions")


Environment ready: 120 actions


In [ ]:
class DuelingDQN(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=256):
        super().__init__()
        self.feature = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.value = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)
        )
        self.advantage = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, action_dim)
        )

    def forward(self, x):
        features = self.feature(x)
        value = self.value(features)
        advantage = self.advantage(features)
        return value + (advantage - advantage.mean(dim=1, keepdim=True))

class PrioritizedReplayBuffer:
    def __init__(self, capacity, alpha=0.6, beta=0.4, beta_increment=0.001):
        self.capacity = capacity
        self.alpha = alpha
        self.beta = beta
        self.beta_increment = beta_increment
        self.buffer = []
        self.priorities = np.zeros(capacity, dtype=np.float32)
        self.position = 0

    def push(self, transition, td_error):
        priority = (abs(td_error) + 1e-5) ** self.alpha
        if len(self.buffer) < self.capacity:
            self.buffer.append(transition)
        else:
            self.buffer[self.position] = transition
        self.priorities[self.position] = priority
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        buffer_size = len(self.buffer)
        priorities = self.priorities[:buffer_size]
        probs = priorities / priorities.sum()
        indices = np.random.choice(buffer_size, batch_size, p=probs)
        weights = (buffer_size * probs[indices]) ** (-self.beta)
        weights /= weights.max()
        self.beta = min(1.0, self.beta + self.beta_increment)
        batch = [self.buffer[i] for i in indices]
        return batch, indices, torch.FloatTensor(weights)

    def update_priorities(self, indices, td_errors):
        for idx, td_error in zip(indices, td_errors):
            self.priorities[idx] = (abs(td_error) + 1e-5) ** self.alpha

    def __len__(self):
        return len(self.buffer)

class DoubleDQNAgent:
    def __init__(self, state_dim, action_dim, lr=1e-4, gamma=0.99, tau=0.005, buffer_size=50000):
        self.action_dim = action_dim
        self.gamma = gamma
        self.tau = tau

        self.q_network = DuelingDQN(state_dim, action_dim)
        self.target_network = DuelingDQN(state_dim, action_dim)
        self.target_network.load_state_dict(self.q_network.state_dict())

        self.optimizer = torch.optim.Adam(self.q_network.parameters(), lr=lr)
        self.memory = PrioritizedReplayBuffer(buffer_size)

        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.9995

    def select_action(self, state, training=True):
        if training and random.random() < self.epsilon:
            return random.randrange(self.action_dim)
        with torch.no_grad():
            return self.q_network(torch.FloatTensor(state).unsqueeze(0)).argmax().item()

    def store_transition(self, state, action, reward, next_state, done):
        with torch.no_grad():
            current_q = self.q_network(torch.FloatTensor(state).unsqueeze(0))[0, action]
            next_q = self.target_network(torch.FloatTensor(next_state).unsqueeze(0)).max().item()
            td_error = reward + self.gamma * next_q * (1 - done) - current_q.item()
        self.memory.push((state, action, reward, next_state, done), td_error)

    def update(self, batch_size=64):
        if len(self.memory) < batch_size:
            return 0.0

        batch, indices, weights = self.memory.sample(batch_size)
        states = torch.FloatTensor([t[0] for t in batch])
        actions = torch.LongTensor([t[1] for t in batch])
        rewards = torch.FloatTensor([t[2] for t in batch])
        next_states = torch.FloatTensor([t[3] for t in batch])
        dones = torch.FloatTensor([t[4] for t in batch])

        current_q = self.q_network(states).gather(1, actions.unsqueeze(1))

        with torch.no_grad():
            next_actions = self.q_network(next_states).argmax(1)
            next_q = self.target_network(next_states).gather(1, next_actions.unsqueeze(1)).squeeze()
            target_q = rewards + self.gamma * next_q * (1 - dones)

        td_errors = (target_q - current_q.squeeze()).detach().numpy()
        self.memory.update_priorities(indices, td_errors)

        loss = (weights * F.mse_loss(current_q.squeeze(), target_q, reduction='none')).mean()

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 10.0)
        self.optimizer.step()

        for target_param, param in zip(self.target_network.parameters(), self.q_network.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
        return loss.item()

agent = DoubleDQNAgent(state_dim=6, action_dim=120)
print(f"Double DQN ready: {sum(p.numel() for p in agent.q_network.parameters())} params, 120 actions")


Double DQN ready: 148985 params, 120 actions


In [ ]:
def train_double_dqn(env, agent, episodes=12000, curriculum_stages=4):
    episode_rewards = []
    episode_errors = []
    action_counts = np.zeros(agent.action_dim)

    episodes_per_stage = episodes // curriculum_stages
    complexity_schedule = [(2, 4), (3, 4), (4, 4), (4, 4)]

    print(f"Training {episodes} episodes, {curriculum_stages} stages")
    start_time = time.time()

    for episode in range(episodes):
        stage = min(episode // episodes_per_stage, curriculum_stages - 1)
        trotter, nq = complexity_schedule[stage]

        env.num_qubits = nq
        env.trotter_steps = trotter

        state, _ = env.reset()

        #if episode == 0 or episode % episodes_per_stage == 0:
            #print(f"\nDEBUG Episode {episode} (Stage {stage+1}):")
            #print(f"  Circuit: {nq}q, {trotter}T, depth={env.circuit.depth()}, gates={len(env.circuit)}")
            #print(f"  Ideal: {env.ideal:.4f}")
            #test_noisy = executor_noisy(env.circuit, env.observable)
            #print(f"  Single noisy run: {test_noisy:.4f}, error={abs(test_noisy - env.ideal):.4f}")

        action = agent.select_action(state, training=True)
        action_counts[action] += 1

        next_state, reward, done, _, info = env.step(action)



        agent.store_transition(state, action, reward, next_state, done)
        loss = agent.update(batch_size=64)

        episode_rewards.append(reward)
        episode_errors.append(info['error'])

        if (episode + 1) % 500 == 0:
            print(f"Ep {episode+1:5d} | Stage {stage+1} ({trotter}T,{nq}q) | "
                  f"Reward: {np.mean(episode_rewards[-500:]):+.4f} | "
                  f"Error: {np.mean(episode_errors[-500:]):.4f} | "
                  f"ε: {agent.epsilon:.3f}")

    print(f"Complete in {(time.time()-start_time)/60:.1f}m")
    return {'rewards': episode_rewards, 'errors': episode_errors, 'action_counts': action_counts}

results = train_double_dqn(env, agent, episodes=12000, curriculum_stages=4)


Training 12000 episodes, 4 stages
Ep   500 | Stage 1 (2T,4q) | Reward: -2231.4418 | Error: 2231.4418 | ε: 0.050
Ep  1000 | Stage 1 (2T,4q) | Reward: -0.2889 | Error: 0.2889 | ε: 0.050
Ep  1500 | Stage 1 (2T,4q) | Reward: -0.2122 | Error: 0.2122 | ε: 0.050
Ep  2000 | Stage 1 (2T,4q) | Reward: -0.4586 | Error: 0.4586 | ε: 0.050
Ep  2500 | Stage 1 (2T,4q) | Reward: -0.3745 | Error: 0.3745 | ε: 0.050
Ep  3000 | Stage 1 (2T,4q) | Reward: -0.6827 | Error: 0.6827 | ε: 0.050
Ep  3500 | Stage 2 (3T,4q) | Reward: -0.1555 | Error: 0.1555 | ε: 0.050
Ep  4000 | Stage 2 (3T,4q) | Reward: -0.1602 | Error: 0.1602 | ε: 0.050
Ep  4500 | Stage 2 (3T,4q) | Reward: -0.2812 | Error: 0.2812 | ε: 0.050
Ep  5000 | Stage 2 (3T,4q) | Reward: -0.1675 | Error: 0.1675 | ε: 0.050
Ep  5500 | Stage 2 (3T,4q) | Reward: -0.2133 | Error: 0.2133 | ε: 0.050
Ep  6000 | Stage 2 (3T,4q) | Reward: -0.0959 | Error: 0.0959 | ε: 0.050
Ep  6500 | Stage 3 (4T,4q) | Reward: -0.3095 | Error: 0.3095 | ε: 0.050
Ep  7000 | Stage 3 (4T,4

KeyboardInterrupt: 

In [ ]:
import torch

torch.save({
    'q_network': agent.q_network.state_dict(),
    'target_network': agent.target_network.state_dict(),
    'optimizer': agent.optimizer.state_dict(),
    'epsilon': agent.epsilon,
}, '/content/zne_rl_agent_checkpoint.pth')

print("Model checkpoint saved as zne_rl_agent_checkpoint.pth")


Model checkpoint saved as zne_rl_agent_checkpoint.pth


In [ ]:
from google.colab import files
files.download('/content/zne_rl_agent_checkpoint.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import time

def test_agent(env, agent, num_test_episodes=500):
    """
    Test trained agent on 500 UNSEEN circuits (4 qubits only).
    Compare against fixed baseline strategies.
    """
    print("\n" + "="*70)
    print("TESTING TRAINED AGENT ON 500 UNSEEN CIRCUITS")
    print("="*70)

    agent_errors = []
    agent_actions = []
    baseline_standard = []   # [1, 2, 3] Richardson
    baseline_aggressive = [] # [1, 3, 5, 7] Richardson
    baseline_fine = []       # [1.0, 1.1, 1.2] Linear

    agent.epsilon = 0.0

    print(f"\nTesting on 500 circuits (4 qubits, varying complexity)...")
    start_time = time.time()

    for test_ep in range(num_test_episodes):
        # Vary Trotter complexity
        if test_ep < 125:
            trotter = 2
        elif test_ep < 250:
            trotter = 3
        elif test_ep < 375:
            trotter = 4
        else:
            trotter = 4

        env.num_qubits = 4
        env.trotter_steps = trotter
        state, _ = env.reset()

        # Agent's learned policy
        agent_action = agent.select_action(state, training=False)
        agent_actions.append(agent_action)

        _, _, _, _, agent_info = env.step(agent_action)
        agent_errors.append(agent_info['error'])

        # Baseline 1: [1, 2, 3] Richardson
        template_idx_std = 9  # [1, 2, 3]
        extrap_idx_std = 3    # Richardson
        action_std = template_idx_std * 4 + extrap_idx_std
        _, _, _, _, info_std = env.step(action_std)
        baseline_standard.append(info_std['error'])

        # Baseline 2: [1, 3, 5, 7] Richardson
        template_idx_agg = 17  # [1, 3, 5, 7]
        extrap_idx_agg = 3
        action_agg = template_idx_agg * 4 + extrap_idx_agg
        _, _, _, _, info_agg = env.step(action_agg)
        baseline_aggressive.append(info_agg['error'])

        # Baseline 3: [1.0, 1.1, 1.2] Linear
        template_idx_fine = 0  # [1.0, 1.1, 1.2]
        extrap_idx_fine = 0    # Linear
        action_fine = template_idx_fine * 4 + extrap_idx_fine
        _, _, _, _, info_fine = env.step(action_fine)
        baseline_fine.append(info_fine['error'])

        if (test_ep + 1) % 100 == 0:
            elapsed = (time.time() - start_time) / 60
            print(f"  {test_ep+1}/500 tested ({elapsed:.1f} min elapsed)")

    test_time = (time.time() - start_time) / 60

    # Statistics
    agent_mean = np.mean(agent_errors)
    agent_std = np.std(agent_errors)
    agent_median = np.median(agent_errors)

    std_mean = np.mean(baseline_standard)
    agg_mean = np.mean(baseline_aggressive)
    fine_mean = np.mean(baseline_fine)

    print("\n" + "="*70)
    print("TEST RESULTS (500 circuits, 4 qubits)")
    print("="*70)
    print(f"Test Duration: {test_time:.1f} minutes")
    print(f"\nMean Error (± std | median):")
    print(f"  Trained Agent:           {agent_mean:.4f} ± {agent_std:.4f} | {agent_median:.4f}")
    print(f"  Baseline [1,2,3]+Rich:   {std_mean:.4f}")
    print(f"  Baseline [1,3,5,7]+Rich: {agg_mean:.4f}")
    print(f"  Baseline [1.0,1.1,1.2]+Lin: {fine_mean:.4f}")

    vs_standard = (std_mean - agent_mean) / std_mean * 100
    vs_aggressive = (agg_mean - agent_mean) / agg_mean * 100
    vs_fine = (fine_mean - agent_mean) / fine_mean * 100

    print(f"\nAgent Improvement:")
    print(f"  vs [1,2,3]+Rich:   {vs_standard:+.1f}%")
    print(f"  vs [1,3,5,7]+Rich: {vs_aggressive:+.1f}%")
    print(f"  vs [1.0,1.1,1.2]+Lin: {vs_fine:+.1f}%")

    # Top actions
    action_usage = np.zeros(agent.action_dim)
    for a in agent_actions:
        action_usage[a] += 1

    top_5 = np.argsort(action_usage)[-5:][::-1]

    print(f"\nTop 5 Actions Used by Agent:")
    for i, act_idx in enumerate(top_5, 1):
        template_idx = act_idx // 4
        extrap_idx = act_idx % 4
        factors = env.noisefactor_options[template_idx]
        extrap = env.extrapolators[extrap_idx]
        count = int(action_usage[act_idx])
        pct = count / num_test_episodes * 100
        print(f"  {i}. Action {act_idx}: {factors} + {extrap} ({count} uses, {pct:.1f}%)")

    print("="*70)

    return {
        'agent_errors': agent_errors,
        'agent_actions': agent_actions,
        'baseline_standard': baseline_standard,
        'baseline_aggressive': baseline_aggressive,
        'baseline_fine': baseline_fine,
        'agent_mean': agent_mean,
        'std_mean': std_mean,
        'agg_mean': agg_mean,
        'fine_mean': fine_mean
    }

# Run test after training completes
test_results = test_agent(env, agent, num_test_episodes=500)



TESTING TRAINED AGENT ON 500 UNSEEN CIRCUITS

Testing on 500 circuits (4 qubits, varying complexity)...
  100/500 tested (2.4 min elapsed)
  200/500 tested (5.4 min elapsed)
  300/500 tested (8.8 min elapsed)
  400/500 tested (12.5 min elapsed)
  500/500 tested (16.2 min elapsed)

TEST RESULTS (500 circuits, 4 qubits)
Test Duration: 16.2 minutes

Mean Error (± std | median):
  Trained Agent:           0.1595 ± 0.0511 | 0.1714
  Baseline [1,2,3]+Rich:   0.0997
  Baseline [1,3,5,7]+Rich: 0.1737
  Baseline [1.0,1.1,1.2]+Lin: 0.2930

Agent Improvement:
  vs [1,2,3]+Rich:   -60.0%
  vs [1,3,5,7]+Rich: +8.2%
  vs [1.0,1.1,1.2]+Lin: +45.6%

Top 5 Actions Used by Agent:
  1. Action 117: [1.0, 3.5, 6.0] + polynomial (250 uses, 50.0%)
  2. Action 61: [1.0, 4.0, 5.0] + polynomial (250 uses, 50.0%)
  3. Action 119: [1.0, 3.5, 6.0] + richardson (0 uses, 0.0%)
  4. Action 43: [1, 2, 4] + richardson (0 uses, 0.0%)
  5. Action 31: [1.0, 2.2, 2.7] + richardson (0 uses, 0.0%)
